In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
today=str(datetime.datetime.today().strftime('%d_%m_%Y'))

In [3]:
month=str(datetime.datetime.today().strftime('%Y%m'))

In [4]:
mjr = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\Major_Incidents\Mjr_INC_202104.csv')
insto_mjr = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\insto_mjr_inc\INSTO_mjr_inc_202104.csv')
mjr_new = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\Major_Incidents\Mjr_INC_202104.csv')
domain_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\model1_mapping.csv')

In [5]:
##compute mttr for all domain except institution
mjr = mjr[((mjr['priority']==1) | (mjr['priority']==2))]
mjr = mjr[mjr['major_incident_state']=='accepted']
mjr['resolved_at'] = pd.to_datetime(mjr['resolved_at'],format='%Y-%m-%d %H:%M:%S')
mjr['created_on'] = pd.to_datetime(mjr['created_on'],format='%Y-%m-%d %H:%M:%S')
mjr = mjr[mjr['created_on']>=pd.to_datetime('2019-10-01',format='%Y-%m-%d')]
mjr['res_time'] = mjr['resolved_at'] - mjr['created_on']
mjr['created_month'] = mjr.created_on.apply(lambda x: x.replace(day=1).date())

In [6]:
domain_map = domain_map.loc[domain_map.notnull().sum(1).groupby(domain_map.Name).idxmax()]

In [7]:
mjr['business_service_name'] = mjr['business_service_name'].str.strip('[PROD]')
mjr['business_service_name'] = mjr['business_service_name'].str.strip()

In [8]:
mjr = mjr.merge(domain_map,how='left',left_on=['business_service_name'],right_on=['Name'])
mjr = mjr[~(mjr['Domain']=='Institutional')]

In [9]:
res_time_sum_dm = mjr.groupby(['Domain','business_service_name','created_month'])['res_time'].sum().reset_index(name='sum')

In [10]:
mjr_incdnt_mix_dm = mjr.groupby(['Domain','business_service_name','created_month'])['number'].size().reset_index(name='count')


In [11]:
mttr_dm = res_time_sum_dm.merge(mjr_incdnt_mix_dm, on=['Domain','business_service_name','created_month'],how='left')
mttr_dm['mttr'] = mttr_dm['sum']/mttr_dm['count']
mttr_dm['mttr'] = (mttr_dm['mttr'].dt.total_seconds()/3600).round(2)
mttr_dm = mttr_dm.drop(['count'],axis=1)

In [12]:
res_time_sum_dy = mjr.groupby(['Domain','business_service_name'])['res_time'].sum().reset_index(name='sum')
mjr_incdnt_mix_dy = mjr.groupby(['Domain','business_service_name'])['number'].size().reset_index(name='count')
mttr_dy = res_time_sum_dy.merge(mjr_incdnt_mix_dy, on=['Domain','business_service_name'],how='left')
mttr_dy['mttr'] = mttr_dy['sum']/mttr_dy['count']
mttr_dy['mttr'] = (mttr_dy['mttr'].dt.total_seconds()/3600).round(2)
mttr_dy = mttr_dy.drop(['sum'],axis=1)

In [13]:
res_time_sum_am = mjr.groupby(['created_month'])['res_time'].sum().reset_index(name='sum')
mjr_incdnt_mix_am = mjr.groupby(['created_month'])['number'].size().reset_index(name='count')
mttr_am = res_time_sum_am.merge(mjr_incdnt_mix_am, on=['created_month'],how='left')

In [14]:
mjr_new = mjr_new.drop(['priority'],axis=1)

In [15]:
mjr_new['business_service_name'] = mjr_new['business_service_name'].str.strip('[PROD]')
mjr_new['business_service_name'] = mjr_new['business_service_name'].str.strip()

In [16]:
insto_mjr = insto_mjr[(insto_mjr['sgm_service_group']=='Asia, Europe, America') | (insto_mjr['sgm_service_group']=='Institutional')]
insto_mjr['cio_begin'] = pd.to_datetime(insto_mjr['cio_begin'],format='%Y-%m-%d %H:%M:%S')
insto_mjr['cio_end'] = pd.to_datetime(insto_mjr['cio_end'],format='%Y-%m-%d %H:%M:%S')


In [17]:
domain_map = domain_map.loc[domain_map.notnull().sum(1).groupby(domain_map.Name).idxmax()]
mjr_new = mjr_new.merge(domain_map,how='left',left_on=['business_service_name'],right_on=['Name'])

In [18]:
#insto_mjr['Domain'] = 'Institutional'
insto_mjr['res_time'] = insto_mjr['cio_end'] - insto_mjr['cio_begin']

In [19]:
insto_mjr = insto_mjr.merge(mjr_new,how='left',left_on=['cio_task_number'],right_on=['number'])
insto_mjr = insto_mjr[(insto_mjr['major_incident_state']=='accepted') | (insto_mjr['major_incident_state']=='canceled')]
insto_mjr = insto_mjr[((insto_mjr['priority']==1) | (insto_mjr['priority']==2))]
insto_mjr = insto_mjr.sort_values(['cio_task_number','res_time'],ascending=False)
insto_mjr = insto_mjr.drop_duplicates(subset=['cio_task_number'],keep='first')
insto_mjr = insto_mjr[insto_mjr['cio_begin']>=pd.to_datetime('2019-10-01',format='%Y-%m-%d')]
insto_mjr['created_month'] = insto_mjr.cio_begin.apply(lambda x: x.replace(day=1).date())

In [20]:
res_time_sum_dm = insto_mjr.groupby(['Domain','business_service_name','created_month'])['res_time'].sum().reset_index(name='sum')
mjr_incdnt_mix_dm = insto_mjr.groupby(['Domain','business_service_name','created_month'])['cio_task_number'].size().reset_index(name='count')
insto_mttr_dm = res_time_sum_dm.merge(mjr_incdnt_mix_dm, on=['Domain','business_service_name','created_month'],how='left')
insto_mttr_dm['mttr'] = insto_mttr_dm['sum']/insto_mttr_dm['count']
insto_mttr_dm['mttr'] = (insto_mttr_dm['mttr'].dt.total_seconds()/3600).round(2)
insto_mttr_dm = insto_mttr_dm.drop(['sum'],axis=1)

In [21]:
res_time_sum_dy = insto_mjr.groupby(['Domain','business_service_name'])['res_time'].sum().reset_index(name='sum')
mjr_incdnt_mix_dy = insto_mjr.groupby(['Domain','business_service_name'])['cio_task_number'].size().reset_index(name='count')
insto_mttr_dy = res_time_sum_dy.merge(mjr_incdnt_mix_dy, on=['Domain','business_service_name'],how='left')
insto_mttr_dy['mttr'] = insto_mttr_dy['sum']/insto_mttr_dy['count']
insto_mttr_dy['mttr'] = (insto_mttr_dy['mttr'].dt.total_seconds()/3600).round(2)
insto_mttr_dy = insto_mttr_dy.drop(['sum'],axis=1)

In [22]:
insto_res_time_sum_am = insto_mjr.groupby(['created_month'])['res_time'].sum().reset_index(name='sum')
insto_mjr_incdnt_mix_am = insto_mjr.groupby(['created_month'])['cio_task_number'].size().reset_index(name='count')
insto_mttr_am = insto_res_time_sum_am.merge(insto_mjr_incdnt_mix_am, on=['created_month'],how='left')

In [23]:
res_time_sum_ay = mjr['res_time'].sum() + insto_mjr['res_time'].sum()
mjr_incdnt_mix_ay = len(mjr['number']) + len(insto_mjr['cio_task_number'])
mttr_ay = res_time_sum_ay/mjr_incdnt_mix_ay
mttr_ay = round((mttr_ay.total_seconds()/3600),2)
mttr_ay = pd.DataFrame(data={'Domain':['ANZ Overall'],'mttr':[mttr_ay]})

In [24]:
mttr_y = mttr_ay.append(mttr_dy).append(insto_mttr_dy)
mttr_y['DataLoadTimestamp'] = pd.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [25]:
mttr_m = mttr_am.merge(insto_mttr_am,on=['created_month'],how='outer')
mttr_m['sum'] = mttr_m['sum_x'] + mttr_m['sum_y']
mttr_m['count'] = mttr_m['count_x'] + mttr_m['count_y']
mttr_m['mttr'] = mttr_m['sum']/mttr_m['count']
mttr_m['mttr'] = (mttr_m['mttr'].dt.total_seconds()/3600).round(2)
mttr_m['Domain'] = 'ANZ Overall'
mttr_m = mttr_m[['Domain','created_month','count','mttr']]
mttr_m = mttr_m.append(mttr_dm).append(insto_mttr_dm)
mttr_m['DataLoadTimestamp'] = pd.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [26]:
mttr_m = mttr_m[~(mttr_m['Domain']=='ANZ Overall')]
mttr_y = mttr_y[~(mttr_y['Domain']=='ANZ Overall')]

In [27]:
mttr_m = mttr_m[['Domain','business_service_name','created_month','count','mttr']]

In [28]:
mttr_y = mttr_y[['Domain','business_service_name','count','mttr']]

In [29]:
mttr_y['count'] = mttr_y['count'].fillna(1)
mttr_m['count'] = mttr_m['count'].fillna(1)
mttr_y['count'] = mttr_y['count'].astype(int)
mttr_m['count'] = mttr_m['count'].astype(int)

In [30]:
mttr_y.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\mttr_yearly_'+today+'.csv',index=None)

In [31]:
mttr_m.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\mttr_monthly_'+today+'.csv',index=None)